In [39]:
import pandas as pd
import numpy as np
import logging
import json
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
import logging

In [1]:
# We will import the Json train and test files.

In [6]:
# Opening JSON file
f = "/Users/Alejandro/Desktop/MASTER/Machine learning 23/train.json"

with open(f, 'r') as file:
    json_data = json.load(file)

# we convert it into a pd dataframe
train = pd.DataFrame(json_data).fillna("")
train.tail(10)

# We visualize some lines of the training data set in a pandas dataframe to have an insight.

,ENTRYTYPE,title,editor,year,publisher,author,abstract
65904,inproceedings,Coreference in Knowledge Editing,,1998,,"[van Deemter, Kees, Power, Richard]",
65905,proceedings,COLING 1965,,1965,,,
65906,inproceedings,Precision-focused Textual Inference,,2007,Association for Computational Linguistics,"[Bobrow, Daniel, Crouch, Dick, King, Tracy Hol...",
65907,inproceedings,Generative Knowledge Graph Construction: A Review,,2022,Association for Computational Linguistics,"[Ye, Hongbin, Zhang, Ningyu, Chen, Hui, Chen, ...",Generative Knowledge Graph Construction (KGC) ...
65908,inproceedings,Mind the Gap: Data Enrichment in Dependency Pa...,,2018,Association for Computational Linguistics,"[Droganova, Kira, Ginter, Filip, Kanerva, Jenn...","In this paper, we focus on parsing rare and no..."
65909,inproceedings,Optimizing the weighted sequence alignment alg...,,2022,Association for Computational Linguistics,"[Janicki, Maciej]",We present an optimized implementation of the ...
65910,proceedings,Proceedings of the 25th Conference on Computat...,"[Bisazza, Arianna, Abend, Omri]",2021,Association for Computational Linguistics,,
65911,article,A Large-Scale Pseudoword-Based Evaluation Fram...,,2014,MIT Press,"[Pilehvar, Mohammad Taher, Navigli, Roberto]",
65912,inproceedings,CIST System for CL-SciSumm 2016 Shared Task,,2016,,"[Li, Lei, Mao, Liyuan, Zhang, Yazhao, Chi, Jun...",
65913,inproceedings,Ontology Engineering and Knowledge Extraction ...,,2009,Association for Computational Linguistics,"[Trapman, Jantine, Monachesi, Paola]",


In [17]:
# We calulate summary statistics to know more about our predicted variable

train['year'] = pd.to_numeric(train['year'], errors='coerce')
print(train['year'].dtype)
year = train['year']

summary_stats = year.describe()
print(summary_stats)

mean_value = year.mean()

std_dev = year.std()

se = std_dev / np.sqrt(len(year))

max_value = year.max()
min_value = year.min()

q1 = year.quantile(0.25)
median_value = year.median()
q3 = year.quantile(0.75)

# Display the results
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)
print("Standard Error:", se)
print("Minimum:", min_value)
print("Maximum:", max_value)
print("25th Percentile (Q1):", q1)
print("Median (50th Percentile):", median_value)
print("75th Percentile (Q3):", q3)

# This was to check if the algo was working compare to the std.

int64
count    46139.000000
mean      2012.572141
std         10.164280
min       1952.000000
25%       2008.000000
50%       2016.000000
75%       2020.000000
max       2023.000000
Name: year, dtype: float64
Mean: 2012.5721407052602
Standard Deviation: 10.164280477955465
Standard Error: 0.04731976202929839
Minimum: 1952
Maximum: 2023
25th Percentile (Q1): 2008.0
Median (50th Percentile): 2016.0
75th Percentile (Q3): 2020.0


In [18]:
print(train.columns) # We check the name of the columns in the environment and then check because there seem to be many missing information

column_name = 'abstract'  
if column_name in train.columns:
    empty_string_count = (train[column_name] == '').sum()
    print(f"Number of empty string cells in '{column_name}': {empty_string_count}")
else:
    print(f"Column '{column_name}' not found in the DataFrame.")



Index(['ENTRYTYPE', 'title', 'editor', 'year', 'publisher', 'author',
       'abstract'],
      dtype='object')
Number of empty string cells in 'abstract': 23528


In [32]:
# Now we check the dimensions of the pandas data frame because it looks like a problem of missing data in some columns.

print(train.shape) 
print()
print("Percentage of column 'abstract' missing values:", round((33642*100)/65914, 2),"%")

# There is more than 50% of missing data for the abstract column

(46139, 5)

Percentage of column 'abstract' missing values: 51.04 %


In [33]:
#Now we do the same for the rest of the columns

column_name = 'editor'  # Replace with your column name
if column_name in train.columns:
    empty_string_count = (train[column_name] == '').sum()
    print(f"Number of empty string cells in '{column_name}': {empty_string_count}")
else:
    print(f"Column '{column_name}' not found in the DataFrame.")
print("Percentage of column 'editor' missing values:", round((64438*100)/65914, 2),"%")


print()
column_name = 'publisher'  # Replace with your column name
if column_name in train.columns:
    empty_string_count = (train[column_name] == '').sum()
    print(f"Number of empty string cells in '{column_name}': {empty_string_count}")
else:
    print(f"Column '{column_name}' not found in the DataFrame.")
print("Percentage of column 'publisher' missing values:", round((8201*100)/65914, 2),"%")


print()
column_name = 'author'  # Replace with your column name
if column_name in train.columns:
    empty_string_count = (train[column_name] == '').sum()
    print(f"Number of empty string cells in '{column_name}': {empty_string_count}")
else:
    print(f"Column '{column_name}' not found in the DataFrame.")
print("Percentage of column 'author' missing values:", round((2399*100)/65914, 2),"%")

print()
column_name = 'year'  # Replace with your column name
if column_name in train.columns:
    empty_string_count = (train[column_name] == '').sum()
    print(f"Number of empty string cells in '{column_name}': {empty_string_count}")
else:
    print(f"Column '{column_name}' not found in the DataFrame.")
print("Percentage of column 'author' missing values:", round((0*100)/65914, 2),"%")

# Check how round function was used.


Column 'editor' not found in the DataFrame.
Percentage of column 'editor' missing values: 97.76 %

Number of empty string cells in 'publisher': 5686
Percentage of column 'publisher' missing values: 12.44 %

Number of empty string cells in 'author': 1728
Percentage of column 'author' missing values: 3.64 %

Number of empty string cells in 'year': 0
Percentage of column 'author' missing values: 0.0 %


In [34]:
# There so many unavailable data for the 'editor' column, that we delete it from our pd data frame.

column_to_remove = 'editor'
train = train.drop(column_to_remove, axis=1)

pd.DataFrame(train)

KeyError: "['editor'] not found in axis"

In [30]:
# Lets do the same with abstract to try to improve our MAE.

column_to_remove2 = 'abstract'
train = train.drop(column_to_remove2, axis=1)

pd.DataFrame(train)

KeyError: "['abstract'] not found in axis"

In [35]:
# Opening JSON file
f = "/Users/Alejandro/Desktop/MASTER/Machine learning 23/test.json"

with open(f, 'r') as file:
    json_data2 = json.load(file)

# we convert it into a pd dataframe
test = pd.DataFrame(json_data2).fillna("")

# We some lines of the testing data set to have an insight, and we see there is the predictor column missing so check.

In [29]:
# Lets delete the column 'editor' for the testing set as well.
# Delete editor column

print(test)

column_to_remove = 'editor'
test = test.drop(column_to_remove, axis=1)

pd.DataFrame(test)  

# You forgot to delete the abstract column for the test set.

           ENTRYTYPE                                              title  \
0      inproceedings          Learning to lemmatise Polish noun phrases   
1      inproceedings  The Treebanked Conspiracy. Actors and Actions ...   
2      inproceedings       Linguistic structure and machine translation   
3      inproceedings  NSEmo at EmoInt-2017: An Ensemble to Predict E...   
4      inproceedings     Explaining data using causal Bayesian networks   
...              ...                                                ...   
21967  inproceedings  Scalable Font Reconstruction with Dual Latent ...   
21968  inproceedings  UniGeo: Unifying Geometry Logical Reasoning vi...   
21969  inproceedings  Gradient-guided Unsupervised Lexically Constra...   
21970  inproceedings  Semantically Constrained Multilayer Annotation...   
21971  inproceedings  Annotating Students' Understanding of Science ...   

      editor                                       publisher  \
0                  Association for 

,ENTRYTYPE,title,publisher,author,abstract
0,inproceedings,Learning to lemmatise Polish noun phrases,Association for Computational Linguistics,"['Radziszewski, Adam']",
1,inproceedings,The Treebanked Conspiracy. Actors and Actions ...,,"['Passarotti, Marco', 'González Saavedra, Berta']",
2,inproceedings,Linguistic structure and machine translation,,"['Lamb, Sydney M.']",If one understands the nature of linguistic st...
3,inproceedings,NSEmo at EmoInt-2017: An Ensemble to Predict E...,Association for Computational Linguistics,"['Madisetty, Sreekanth', 'Desarkar, Maunendra ...","In this paper, we describe a method to predict..."
4,inproceedings,Explaining data using causal Bayesian networks,Association for Computational Linguistics,"['Sevilla, Jaime']",I introduce Causal Bayesian Networks as a form...
...,...,...,...,...,...
21967,inproceedings,Scalable Font Reconstruction with Dual Latent ...,Association for Computational Linguistics,"['Srivatsan, Nikita', 'Wu, Si', 'Barron, Jonat...",We propose a deep generative model that perfor...
21968,inproceedings,UniGeo: Unifying Geometry Logical Reasoning vi...,Association for Computational Linguistics,"['Chen, Jiaqi', 'Li, Tong', 'Qin, Jinghui', 'L...",Geometry problem solving is a well-recognized ...
21969,inproceedings,Gradient-guided Unsupervised Lexically Constra...,Association for Computational Linguistics,"['Sha, Lei']",Lexically constrained generation requires the ...
21970,inproceedings,Semantically Constrained Multilayer Annotation...,Association for Computational Linguistics,"['Prange, Jakob', 'Schneider, Nathan', 'Abend,...",We propose a coreference annotation scheme as ...


In [36]:
# Now we delete the 'abstract' column in the test set.

print(test)

column_to_remove = 'abstract'
test = test.drop(column_to_remove, axis=1)

pd.DataFrame(test)  


           ENTRYTYPE                                              title  \
0      inproceedings          Learning to lemmatise Polish noun phrases   
1      inproceedings  The Treebanked Conspiracy. Actors and Actions ...   
2      inproceedings       Linguistic structure and machine translation   
3      inproceedings  NSEmo at EmoInt-2017: An Ensemble to Predict E...   
4      inproceedings     Explaining data using causal Bayesian networks   
...              ...                                                ...   
21967  inproceedings  Scalable Font Reconstruction with Dual Latent ...   
21968  inproceedings  UniGeo: Unifying Geometry Logical Reasoning vi...   
21969  inproceedings  Gradient-guided Unsupervised Lexically Constra...   
21970  inproceedings  Semantically Constrained Multilayer Annotation...   
21971  inproceedings  Annotating Students' Understanding of Science ...   

      editor                                       publisher  \
0                  Association for 

,ENTRYTYPE,title,editor,publisher,author
0,inproceedings,Learning to lemmatise Polish noun phrases,,Association for Computational Linguistics,"[Radziszewski, Adam]"
1,inproceedings,The Treebanked Conspiracy. Actors and Actions ...,,,"[Passarotti, Marco, González Saavedra, Berta]"
2,inproceedings,Linguistic structure and machine translation,,,"[Lamb, Sydney M.]"
3,inproceedings,NSEmo at EmoInt-2017: An Ensemble to Predict E...,,Association for Computational Linguistics,"[Madisetty, Sreekanth, Desarkar, Maunendra San..."
4,inproceedings,Explaining data using causal Bayesian networks,,Association for Computational Linguistics,"[Sevilla, Jaime]"
...,...,...,...,...,...
21967,inproceedings,Scalable Font Reconstruction with Dual Latent ...,,Association for Computational Linguistics,"[Srivatsan, Nikita, Wu, Si, Barron, Jonathan, ..."
21968,inproceedings,UniGeo: Unifying Geometry Logical Reasoning vi...,,Association for Computational Linguistics,"[Chen, Jiaqi, Li, Tong, Qin, Jinghui, Lu, Pan,..."
21969,inproceedings,Gradient-guided Unsupervised Lexically Constra...,,Association for Computational Linguistics,"[Sha, Lei]"
21970,inproceedings,Semantically Constrained Multilayer Annotation...,,Association for Computational Linguistics,"[Prange, Jakob, Schneider, Nathan, Abend, Omri]"


In [10]:
# Validation set
from sklearn.model_selection import train_test_split


train, validation = train_test_split(train, train_size = 0.70, random_state=42)       # We create a validation set and see it on a pandas data frame
pd.DataFrame(validation)


,ENTRYTYPE,title,editor,year,publisher,author,abstract
12680,inproceedings,Question-Answering Based on Virtually Integrat...,,2003,Association for Computational Linguistics,"['Choi, Key-Sun', 'Kim, Jae-Ho', 'Miyazaki, Ma...",
17292,inproceedings,AMI&ERIC: How to Learn with Naive Bayes and Pr...,,2013,Association for Computational Linguistics,"['Dermouche, Mohamed', 'Khouas, Leila', 'Velci...",
33265,inproceedings,Inducing Gazetteers for Named Entity Recogniti...,,2008,Association for Computational Linguistics,"[""Kazama, Jun'ichi"", 'Torisawa, Kentaro']",
52850,inproceedings,Leveraging Explicit Lexico-logical Alignments ...,,2022,Association for Computational Linguistics,"['Sun, Runxin', 'He, Shizhu', 'Zhu, Chong', 'H...",Text-to-SQL aims to parse natural language que...
2298,inproceedings,CLAM: Quickly deploy NLP command-line tools on...,,2014,Dublin City University and Association for Com...,"['van Gompel, Maarten', 'Reynaert, Martin']",
...,...,...,...,...,...,...,...
6236,inproceedings,Deep Span Representations for Named Entity Rec...,,2023,Association for Computational Linguistics,"['Zhu, Enwei', 'Liu, Yiyang', 'Li, Jinpeng']",Span-based models are one of the most straight...
63616,inproceedings,Comparing a Hand-crafted to an Automatically G...,,2019,"Incoma Ltd., Shoumen, Bulgaria","['Mouratidis, Despoina', 'Kermanidis, Katia Li...",The automatic evaluation of machine translatio...
6396,inproceedings,Thai Stock News Sentiment Classification using...,,2015,,"['Netisopakul, Ponrudee', 'Chattupan, Apinan']",
28216,inproceedings,Effects of Empty Categories on Machine Transla...,,2010,Association for Computational Linguistics,"['Chung, Tagyoung', 'Gildea, Daniel']",


In [27]:
test['title'] = test['title'].astype(str)
test['author'] = test['author'].astype(str)

featurizer = ColumnTransformer(
        transformers=[("title", CountVectorizer(), "title"),
                ("author", CountVectorizer(), "author"),
                ("ENTRYTYPE", CountVectorizer(), "ENTRYTYPE"),
                ("publisher", CountVectorizer(), "publisher")],
        remainder='drop')

dummy = make_pipeline(featurizer, DummyRegressor(strategy = 'mean'))
ridge = make_pipeline(featurizer, Ridge())
logging.info("Fitting models")
dummy.fit(train.drop('year', axis=1), train['year'].values)
ridge.fit(train.drop('year', axis=1), train['year'].values)
err = mean_absolute_error(validation['year'].values, ridge.predict(validation.drop('year', axis=1)))
print(validation.dtypes)
print(err)

err = mean_absolute_error(test['year'].values, ridge.predict(test.drop('year', axis=1)))

ValueError: X has 6 features, but ColumnTransformer is expecting 4 features as input.

In [37]:
#implement one hot encoding for entrytype and publisher although publisher has more unique entries to one hot encode
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
import logging

train['title'] = train['title'].astype(str)
train['author'] = train['author'].astype(str)
train['year'] = pd.to_numeric(train['year'], errors='coerce')

validation['title'] = validation['title'].astype(str)
validation['author'] = validation['author'].astype(str)
validation['year'] = pd.to_numeric(validation['year'], errors='coerce')

# Assuming 'ENTRYTYPE' and 'publisher' are categorical columns
categorical_columns = ['ENTRYTYPE', 'publisher']

# Create a column transformer with one-hot encoding for categorical variables
featurizer = ColumnTransformer(
    transformers=[
        ("title", TfidfVectorizer(), "title"),
        ("author", TfidfVectorizer(), "author"),
        ("categorical", OneHotEncoder(), categorical_columns)
    ],
    remainder='drop'
)

dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
ridge = make_pipeline(featurizer, Ridge())

logging.info("Fitting models")
dummy.fit(train.drop('year', axis=1), train['year'].values)
ridge.fit(train.drop('year', axis=1), train['year'].values)

err = mean_absolute_error(validation['year'].values, ridge.predict(validation.drop('year', axis=1)))
print(validation.dtypes)
print(err)


ValueError: X has 6 features, but ColumnTransformer is expecting 4 features as input.

In [24]:
# Assuming only 'ENTRYTYPE' is a categorical column
categorical_columns = ['ENTRYTYPE']

# Create a column transformer with one-hot encoding for 'ENTRYTYPE' only
featurizer = ColumnTransformer(
    transformers=[
        ("title", TfidfVectorizer(), "title"),
        ("author", TfidfVectorizer(), "author"),
        ("ENTRYTYPE", OneHotEncoder(), categorical_columns),
        ("publisher", TfidfVectorizer(), "publisher")],
        remainder='drop')

dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
ridge = make_pipeline(featurizer, Ridge())

logging.info("Fitting models")
dummy.fit(train.drop('year', axis=1), train['year'].values)
ridge.fit(train.drop('year', axis=1), train['year'].values)

err = mean_absolute_error(validation['year'].values, ridge.predict(validation.drop('year', axis=1)))
print(validation.dtypes)
print(err)

NameError: name 'TfidfVectorizer' is not defined

In [38]:
train_sample = train.sample(frac = 1)

In [43]:
%%time

# Constants
TITLE_COLUMN = 'title'
TARGET_COLUMN = 'year'

# Feature engineering
categorical_columns = ['ENTRYTYPE']

featurizer = ColumnTransformer(
    transformers=[
        ("title", TfidfVectorizer(), "title"),
        ("author", TfidfVectorizer(), "author"),
        ("ENTRYTYPE", OneHotEncoder(), categorical_columns),
        ("publisher", TfidfVectorizer(), "publisher")],
        remainder='drop')

dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
ridge = make_pipeline(featurizer, Ridge())

logging.info("Fitting models")
dummy.fit(train.drop('year', axis=1), train['year'].values)
ridge.fit(train.drop('year', axis=1), train['year'].values)

# Create pipelines
random_forest = make_pipeline(featurizer, RandomForestRegressor(n_estimators = 500, random_state=123, n_jobs = -1, max_depth = 50))

# Assuming your data is split into features and target
X = train_sample.drop(TARGET_COLUMN, axis=1)
y = train_sample[TARGET_COLUMN]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
random_forest.fit(X_train, y_train)

# Make predictions on the test set
predictions = random_forest.predict(X_test)

# Evaluate the model
err = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error on Test Set: {err}')


Mean Absolute Error on Test Set: 4.399318304314843
Wall time: 22h 33min 4s


In [ ]:
%%time #4.86     5.09  5.10   

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

# Constants
TITLE_COLUMN = 'title'
TARGET_COLUMN = 'year'

# Feature engineering
categorical_columns = ['ENTRYTYPE']

featurizer = ColumnTransformer(
    transformers=[
        ("title", TfidfVectorizer(), "title"),
        ("author", TfidfVectorizer(), "author"),
        ("ENTRYTYPE", OneHotEncoder(), categorical_columns),
        ("publisher", TfidfVectorizer(), "publisher")],
        remainder='drop')

dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
ridge = make_pipeline(featurizer, Ridge())

logging.info("Fitting models")
dummy.fit(train.drop('year', axis=1), train['year'].values)
ridge.fit(train.drop('year', axis=1), train['year'].values)

# Create pipelines
random_forest = make_pipeline(featurizer, RandomForestRegressor(n_estimators = 500, 
                                                                random_state = 123, n_jobs = -1, min_samples_split = 0.1, max_features = 'auto', max_depth = 18))

# Assuming you have your features (X) and target variable (y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the RandomForestRegressor with some hyperparameters to tune
rf_regressor = RandomForestRegressor()

# Define the hyperparameter grid to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Use GridSearchCV to search for the best hyperparameters
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=param_grid,
                           scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Print the best cross-validated score
best_score = grid_search.best_score_
print("Best Cross-Validated Score (negative mean squared error):", best_score)

# Access the complete results, including all hyperparameter combinations
cv_results = grid_search.cv_results_
print("Complete Cross-Validation Results:", cv_results)
                              
                              

# Assuming your data is split into features and target
X = train.drop(TARGET_COLUMN, axis=1)
y = train[TARGET_COLUMN] 

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Fit the model
random_forest.fit(X_train, y_train)

# Make predictions on the test set
predictions = random_forest.predict(X_test)

# Evaluate the model
err = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error on Test Set: {err}')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

# Constants
TITLE_COLUMN = 'title'
TARGET_COLUMN = 'year'

# Feature engineering
categorical_columns = ['ENTRYTYPE']

featurizer = ColumnTransformer(
    transformers=[
        ("title", TfidfVectorizer(), "title"),
        ("author", TfidfVectorizer(), "author"),
        ("ENTRYTYPE", OneHotEncoder(), categorical_columns),
        ("publisher", TfidfVectorizer(), "publisher")],
        remainder='drop')

dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
ridge = make_pipeline(featurizer, Ridge())

logging.info("Fitting models")
dummy.fit(train.drop('year', axis=1), train['year'].values)
ridge.fit(train.drop('year', axis=1), train['year'].values)

# Create pipelines
random_forest = make_pipeline(featurizer, RandomForestRegressor(random_state=42))

# Assuming your data is split into features and target
X = validation_sample.drop(TARGET_COLUMN, axis=1)
y = validation_sample[TARGET_COLUMN]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
random_forest.fit(X_train, y_train)

# Make predictions on the test set
predictions = random_forest.predict(X_test)

# Evaluate the model
err = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error on Test Set: {err}')


Mean Absolute Error on Test Set: 6.594545454545446


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
import logging

# Constants
TITLE_COLUMN = 'title'
TARGET_COLUMN = 'year'

# Feature engineering
categorical_columns = ['ENTRYTYPE']

featurizer = ColumnTransformer(
    transformers=[
        ("title", TfidfVectorizer(), "title"),
        ("author", TfidfVectorizer(), "author"),
        ("ENTRYTYPE", OneHotEncoder(), categorical_columns),
        ("publisher", TfidfVectorizer(), "publisher")],
        remainder='drop')

# Create pipelines
dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
random_forest = make_pipeline(featurizer, RandomForestRegressor())

# Define the parameter grid for Random Forest
param_grid = {
    'randomforestregressor__n_estimators': [50, 100, 200],
    'randomforestregressor__max_depth': [None, 10, 20],
    'randomforestregressor__min_samples_split': [2, 5, 10],
    'randomforestregressor__min_samples_leaf': [1, 2, 4]
}

print('check')

# Perform Grid Search Cross-Validation
grid_search = GridSearchCV(random_forest, param_grid, scoring='neg_mean_absolute_error', cv=5)
grid_search.fit(train.drop(TARGET_COLUMN, axis=1), train[TARGET_COLUMN].values)

# Get the best model and its parameters
best_random_forest_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Log the best parameters
logging.info(f'Best Parameters: {best_params}')

# Evaluate the best model
err = mean_absolute_error(validation_sample[TARGET_COLUMN].values, best_random_forest_model.predict(validation_sample.drop(TARGET_COLUMN, axis=1)))
logging.info(f'Mean Absolute Error on Validation Set: {err}')


check


In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error

# Constants
TITLE_COLUMN = 'title'
TARGET_COLUMN = 'year'

# Feature engineering
featurizer = ColumnTransformer(
    transformers=[(TITLE_COLUMN, CountVectorizer(), TITLE_COLUMN)],
    remainder='drop'
)

# Create pipelines
dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))

# Simplified Random Forest
random_forest = make_pipeline(featurizer, RandomForestRegressor(
    n_estimators=100,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1
))

# Fit the model
random_forest.fit(train.drop(TARGET_COLUMN, axis=1), train[TARGET_COLUMN].values)

# Evaluate the model
err = mean_absolute_error(validation_sample[TARGET_COLUMN].values, random_forest.predict(validation_sample.drop(TARGET_COLUMN, axis=1)))
print(f'Mean Absolute Error on Validation Set: {err}')


NameError: name 'validation_sample' is not defined

In [ ]:
# number of unique publishers
unique_publishers = train['publisher'].nunique()

print(f"Number of unique publishers: {unique_publishers}")

#number of unique entrytype's
unique_entrytype = train['ENTRYTYPE'].nunique()

print(f"Number of unique entrytype's: {unique_entrytype}")

In [ ]:
#see the number of publishers and the count of them
average_year_per_publisher = train.groupby('publisher')['year'].mean()
count_per_publisher = train.groupby('publisher')['year'].count()

# Create a DataFrame with the results
result_df = pd.DataFrame({
    'average_year': average_year_per_publisher,
    'count': count_per_publisher
}).reset_index()

result_df.head(20)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns  # Seaborn enhances the appearance of Matplotlib plots
import pandas as pd

# Assuming 'train' is your DataFrame and 'column_name' is the column you want to check for outliers
column_name = 'year'

# Create a boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(x=train[column_name])

# Display the plot
plt.title(f'Boxplot for {year}')
plt.show()

In [ ]:
import pandas as pd

# Assuming 'train' is your DataFrame and 'column_name' is the column you want to check for outliers
column_name = 'year'

# Calculate Q1, Q3, and IQR
Q1 = train[column_name].quantile(0.25)
Q3 = train[column_name].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR


# Detect outliers
outliers = train[(train[column_name] < lower_bound) | (train[column_name] > upper_bound)]

# Display the outliers
print(f"Outliers in {column_name}:\n{outliers}")

# Display summary statistics
print(train[column_name].describe())

print(lower_bound)
print(upper_bound)

In [ ]:
#remove outliers below iqr
column_name = 'year'

# Calculate Q1, Q3, and IQR for the 'year' column
Q1 = train[column_name].quantile(0.25)
Q3 = train[column_name].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR

# Remove rows where 'year' is lower than the lower bound
train_filtered = train[train[column_name] >= lower_bound]

# Display information about removed rows
removed_rows = train.shape[0] - train_filtered.shape[0]
print(f"Number of rows removed: {removed_rows}")



In [ ]:
train = train_filtered

In [ ]:
#remove outliers below iqr
column_name = 'year'

# Calculate Q1, Q3, and IQR for the 'year' column
Q1 = validation[column_name].quantile(0.25)
Q3 = validation[column_name].quantile(0.75)
IQR = Q3 - Q1

# Define the lower and upper bounds for outliers
lower_bound = Q1 - 1.5 * IQR

# Remove rows where 'year' is lower than the lower bound
validation_filtered = validation[validation[column_name] >= lower_bound]

# Display information about removed rows
removed_rows = validation.shape[0] - validation_filtered.shape[0]
print(f"Number of rows removed: {removed_rows}")


# Assuming only 'ENTRYTYPE' is a categorical column
categorical_columns = ['ENTRYTYPE']

# Create a column transformer with one-hot encoding for 'ENTRYTYPE' only
featurizer = ColumnTransformer(
    transformers=[
        ("title", TfidfVectorizer(), "title"),
        ("author", TfidfVectorizer(), "author"),
        ("ENTRYTYPE", OneHotEncoder(), categorical_columns),
        ("publisher", CountVectorizer(), "publisher")],
        remainder='drop')

dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
ridge = make_pipeline(featurizer, Ridge())

logging.info("Fitting models")
dummy.fit(train.drop('year', axis=1), train['year'].values)
ridge.fit(train.drop('year', axis=1), train['year'].values)

err = mean_absolute_error(validation['year'].values, ridge.predict(validation.drop('year', axis=1)))
print(validation.dtypes)
print(err)

from sklearn.ensemble import RandomForestRegressor

featurizer = ColumnTransformer(
        transformers=[("title", CountVectorizer(), "title"),
        ("ENTRYTPE", CountVectorizer(), "ENTRYTYPE")]
        remainder='drop')
Random_Forest = make_pipeline(featurizer, RandomForestRegressor())
logging.info("Fitting models")
Random_Forest.fit(train.drop('year', axis=1), train['year'].values)
err = mean_absolute_error(validation['year'].values, Random_Forest.predict(validation.drop('year', axis=1)))
logging.info(f"Mean baseline MAE: {err}")
print(err)
pred = Random_Forest.predict(test)
test['year'] = pred


In [ ]:
#do not run
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
import logging

# Constants
TITLE_COLUMN = 'title'
TARGET_COLUMN = 'year'

# Feature engineering
featurizer = ColumnTransformer(
    transformers=[(TITLE_COLUMN, CountVectorizer(), TITLE_COLUMN)],
    remainder='drop'
)

# Create pipelines
dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
random_forest = make_pipeline(featurizer, RandomForestRegressor())

# Define the parameter grid for Random Forest
param_grid = {
    'randomforestregressor__n_estimators': [50, 100, 200],
    'randomforestregressor__max_depth': [None, 10, 20],
    'randomforestregressor__min_samples_split': [2, 5, 10],
    'randomforestregressor__min_samples_leaf': [1, 2, 4]
}

print('check')

# Perform Grid Search Cross-Validation
grid_search = GridSearchCV(random_forest, param_grid, scoring='neg_mean_absolute_error', cv=5)
grid_search.fit(train.drop(TARGET_COLUMN, axis=1), train[TARGET_COLUMN].values)

# Get the best model and its parameters
best_random_forest_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Log the best parameters
logging.info(f'Best Parameters: {best_params}')

# Evaluate the best model
err = mean_absolute_error(validation[TARGET_COLUMN].values, best_random_forest_model.predict(validation.drop(TARGET_COLUMN, axis=1)))
logging.info(f'Mean Absolute Error on Validation Set: {err}')


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
import logging

featurizer = ColumnTransformer(
        transformers=[("title", CountVectorizer(), "title"),
                ("author", CountVectorizer(), "author"),
                ("ENTRYTYPE", CountVectorizer(), "ENTRYTYPE"),
                ("publisher", CountVectorizer(), "publisher")],
        remainder='drop')
dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
Random_Forest = make_pipeline(featurizer, RandomForestRegressor())
logging.info("Fitting models")
dummy.fit(train.drop('year', axis=1), train['year'].values)
ridge.fit(train.drop('year', axis=1), train['year'].values)
err = mean_absolute_error(validation['year'].values, dummy.predict(validation.drop('year', axis=1)))
print(err)



In [ ]:
def main():
    logging.getLogger().setLevel(logging.INFO)
    logging.info("Loading training/test data")
    train = pd.DataFrame.from_records(json.load(open('train.json'))).fillna("")
    test = pd.DataFrame.from_records(json.load(open('test.json'))).fillna("")
    logging.info("Splitting validation")
    train, val = train_test_split(train, stratify=train['year'], random_state=123)
    featurizer = ColumnTransformer(
        transformers=[("title", CountVectorizer(), "title")],
        remainder='drop')
    dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))
    ridge = make_pipeline(featurizer, Ridge())
    logging.info("Fitting models")
    dummy.fit(train.drop('year', axis=1), train['year'].values)
    ridge.fit(train.drop('year', axis=1), train['year'].values)
    logging.info("Evaluating on validation data")
    err = mean_absolute_error(val['year'].values, dummy.predict(val.drop('year', axis=1)))
    logging.info(f"Mean baseline MAE: {err}")
    err = mean_absolute_error(val['year'].values, ridge.predict(val.drop('year', axis=1)))
    logging.info(f"Ridge regress MAE: {err}")
    logging.info(f"Predicting on test")
    pred = ridge.predict(test)
    test['year'] = pred
    logging.info("Writing prediction file")
    test.to_json("predicted.json", orient='records', indent=2)

main()

In [ ]:
print(validation.dtypes)